#Paquetes necesarios

In [1]:
import cv2  
import math 

from ultralytics import YOLO

Desde cámara, detección con yolov8 y modelo nano

In [15]:
# Carga del modelo
model = YOLO('yolov8n.pt')

# Nombre de las distintas clases
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]


# Captura desde lawebcam
vid = cv2.VideoCapture(0)
  
while(True):      
    # fotograma a fotograma
    ret, img = vid.read()
  
    # si hay imagen válida
    if ret:  
        # Perform inference on an image
        results = model(img, stream=True)
        
        # Para cada detección
        for r in results:
            boxes = r.boxes

            for box in boxes:
                # Contenedor
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values
                
                # Confianza
                confidence = math.ceil((box.conf[0]*100))/100
                print("Confidence --->",confidence)

                # Clase
                cls = int(box.cls[0])
                print("Class name -->", classNames[cls])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255*2:
                    R = 255
                    G = 255
                    B = escala - 255*2
                else:
                    if escala >= 255:
                        R = 255
                        G = escala - 255
                        B = 0
                    else:
                        R = escala
                        G = 0
                        B = 0

                # Dibuja el contenedor y clase
                cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
                cv2.putText(img, classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

        # Muestra fotograma
        cv2.imshow('Vid', img)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()


0: 480x640 (no detections), 45.0ms
Speed: 1.0ms preprocess, 45.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 45.0ms
Speed: 1.0ms preprocess, 45.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 44.0ms
Speed: 1.0ms preprocess, 44.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 42.0ms
Speed: 1.0ms preprocess, 42.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 46.0ms
Speed: 1.0ms preprocess, 46.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 42.0ms
Speed: 1.0ms preprocess, 42.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 41.0ms
Speed: 1.0ms preprocess, 41.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 44.0ms
Speed: 1.0ms preprocess, 44.0ms i

In [11]:
import cv2  
import math 
import os
import numpy as np

from ultralytics import YOLO

# Carga del modelo
#model = YOLO('yolov8n.pt')
model = YOLO('datasets/350_images/runs/detect/train4/weights/best.pt')

# Nombre de las distintas clases
# classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
#               "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
#               "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
#               "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
#               "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
#               "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
#               "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
#               "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
#               "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
#               "teddy bear", "hair drier", "toothbrush"
#               ]

classNames = ["vehicle", "license-plate"]

#carpeta_fotos = "C:\\Users\\miica\\Desktop\\Uni\\VC\\VC_Practica4\\otsedom.github.io\\VC\\P5\\Coches con matriculas\\Coches con matriculas"
carpeta_fotos = "C:\\Users\\miica\\.conda\\envs\\VC_P1\\P5\\datasets\\350_images\\train\\images"

# Lista de fotos de la carpeta
imagenes = os.listdir(carpeta_fotos)

contador = 0
# Itera sobre cada archivo en la carpeta
for img in imagenes:    

    # Verificamos si el archivo es una imagen
    if img.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        contador += 1
        # Construye la ruta completa de la imagen
        ruta_imagen = os.path.join(carpeta_fotos, img)

        # Lee la imagen
        imagen = cv2.imread(ruta_imagen)
        # Obtenemos el alto y el ancho para comprobar que los puntos no se salgan de las dimensiones de la imagen
        alto, ancho = imagen.shape[:2]

        # Perform inference on an image
        results = model(imagen, stream=True)

        # Para cada detección
        for r in results:
            boxes = r.boxes
            
            for box in boxes:

                # Nombre de las clases detectadas, filtra por coche
                boxClassName = classNames[int(box.cls[0])]
                
                if boxClassName == "car" or boxClassName == "truck" or boxClassName == "vehicle":

                    # Contenedor
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values
                    
                    # Puntos intermedios para buscar
                    centroX = int((x1 + x2)/2)
                    centroY = int((y1 + y2)/2) - int((y1 + y2)/8)
                    
                    centroX = max(0, min(centroX, ancho - 1))
                    centroY = max(0, min(centroY, alto - 1))

                    # Region de interes para encontrar la matricula
                    roi = imagen[centroY:y2, x1: x2]

                    # Convertimos la zona de interes a gris
                    imagen_gris = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

                    # Desenfoque Gaussiano para reducir ruido y permitirnos mejorar al deteccion de los bordes de la matricula
                    imagen_gris = cv2.GaussianBlur(imagen_gris, (3,3), 0)
                    
                    # Aplicamos umbral                    
                    blockSize = 15
                    C = 2
                    umbral = cv2.adaptiveThreshold(imagen_gris, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, blockSize=blockSize, C=C)                    

                    # Buscamos los contornos en roi umbralizada
                    contornos, _ = cv2.findContours(umbral, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
                    contornos_validos = {}
                    for contorno in contornos:
                        
                        # Calcula el perimetro del contorno
                        perimetro = cv2.arcLength(contorno, True)

                        # Devuelve los puntos que se aproximan al polígono que crea el contorno
                        approx = cv2.approxPolyDP(contorno, 0.02 * perimetro, True)
                        
                        if len(approx) == 4:
                            
                            # Calcula el area
                            area = cv2.contourArea(contorno)  

                            # Margen de area pequeño para quitar los contornos diminutos                            
                            if 500 < area:
                                
                                diff_x_lado1 = abs(approx[1][0][0] - approx[0][0][0])
                                diff_x_lado2 = abs(approx[2][0][0] - approx[3][0][0])
                                
                                diff_y_lado1 = abs(approx[2][0][1] - approx[1][0][1])
                                diff_y_lado2 = abs(approx[3][0][1] - approx[0][0][1])

                                # Calculamos diferencia de areas para comprobar que sea un rectangulo
                                #area1 = diff_x_lado1 * diff_y_lado1
                                #area2 = diff_x_lado2 * diff_y_lado2
                                #diff_area = abs(area1 - area2)
                                umbral_diferencia_x = 40
                                umbral_diferencia_y = 40
                                
                                if diff_x_lado1 < umbral_diferencia_x and diff_x_lado2 < umbral_diferencia_x:
                                    xText = np.min(approx[:, :, 0])
                                    yText = np.min(approx[:, :, 1])
                                    contornos_validos[area] = approx
                                    #cv2.drawContours(roi, [approx], 0, (0, 255, 0), 2)
                                # Calcula la relación de aspecto que debería acercarse a un valor de 2.0                            
                                # x, y, w, h = cv2.boundingRect(approx)
                                # relacion_aspecto = w / float(h)
                                # if 1.0 < relacion_aspecto < 5:
                                #     relacion_area = area / relacion_aspecto

                                #     if relacion_area < 15000:
                                #         xText = np.min(approx[:, :, 0])
                                #         yText = np.min(approx[:, :, 1])                                    
                                #         cv2.drawContours(roi, [approx], 0, (0, 255, 0), 2)                                        
                                #         cv2.putText(roi, "matricula" , (xText, yText), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    if contornos_validos:
                        mayor_area = max(contornos_validos.keys())
                        contorno_mayor_area = contornos_validos[mayor_area]
                        xText = np.min(contorno_mayor_area[:, :, 0])
                        yText = np.min(contorno_mayor_area[:, :, 1])
                        cv2.drawContours(roi, [contorno_mayor_area], 0, (0, 255, 0), 2)
                        cv2.putText(roi, "matricula" , (xText, yText), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.rectangle(imagen, (x1, centroY), (x2, y2), (255, 255, 255), 3)

                    # Confianza
                    confidence = math.ceil((box.conf[0]*100))/100
                    print("Confidence --->",confidence)

                    # Clase
                    cls = int(box.cls[0])
                    print("Class name -->", classNames[cls])

                    # Convierte identificador numérico de clase a un color RGB
                    escala = int((cls / len(classNames)) * 255 * 3)
                    if escala >= 255*2:
                        R = 255
                        G = 255
                        B = escala - 255*2
                    else:
                        if escala >= 255:
                            R = 255
                            G = escala - 255
                            B = 0
                        else:
                            R = escala
                            G = 0
                            B = 0

                    # Dibuja el contenedor y clase
                    cv2.rectangle(imagen, (x1, y1), (x2, y2), (R, G, B), 3)
                    cv2.putText(imagen, classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

            cv2.putText(imagen, f'Imagen {contador}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            # Muestra la imagen
            cv2.imshow('Imagen', imagen)
            
            # Pasa de foto pulsando la tecla derecha
            if cv2.waitKey(0) == 255 or cv2.waitKey(0) == 83:                
                continue
            
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
                

# Cierra todas las ventanas al finalizar
cv2.destroyAllWindows()


Confidence ---> 0.93
Class name --> vehicle
0: 640x640 1 license-plate, 3 vehicles, 51.0ms
Speed: 3.0ms preprocess, 51.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

Confidence ---> 0.74
Class name --> vehicle
Confidence ---> 0.28
Class name --> vehicle
0: 640x640 2 license-plates, 4 vehicles, 52.0ms
Speed: 2.0ms preprocess, 52.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

Confidence ---> 0.86
Class name --> vehicle
Confidence ---> 0.83
Class name --> vehicle
0: 640x640 2 license-plates, 2 vehicles, 50.0ms
Speed: 2.0ms preprocess, 50.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

Confidence ---> 0.88
Class name --> vehicle
Confidence ---> 0.84
Class name --> vehicle
0: 640x640 2 license-plates, 3 vehicles, 47.0ms
Speed: 2.0ms preprocess, 47.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

Confidence ---> 0.93
Class name --> vehicle
0: 640x640 1 license-plate, 4 vehicles, 51.0ms
Speed: 2.0ms prep

Reconocimiento de caracteres tras instalar pytesseract y tesseract

In [ ]:
# Tesseract
import cv2
import pytesseract

# Previamente debes descargar los ejecutables
# Si la ruta de Tesseract no está en el PATH, ruta al ejecutable
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

# Lenguajes disponibles
print(pytesseract.get_languages(config=''))

#Cargo imagen y ocnvierto a RGB
img = cv2.imread('toy.tif') 
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#Aplica reconocedor a imagen cargada
print(pytesseract.image_to_string(img_rgb))

Reconocimiento decaracteres tras instalar easyocr

In [ ]:
import easyocr

#Carga del modelo de lengua
reader = easyocr.Reader(['es']) 

#Reconocimiento de una imagen
result = reader.readtext('toy.tif')
print(result)

#Con restricción de caracteres reconocibles
#result = reader.readtext('toy.tif', allowlist ='0123456789')

Importa un dataset de Roboflow con 350 coches etiquetados con "vehicle" y "license-plate"

Hay que poner tu API Key para poder descargarlo o entrar en el [dataset](https://public.roboflow.com/object-detection/license-plates-us-eu/3/download/yolov8) de Roboflow, descargarlo como zip y descomprimirlo en el proyecto.

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="YOUR-API-KEY")
project = rf.workspace("samrat-sahoo").project("license-plates-us-eu")
dataset = project.version(3).download("yolov8")

In [ ]:
import cv2  
import math 
import os
import numpy as np
import re

import pytesseract
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

from ultralytics import YOLO

# Carga del modelo
model = YOLO('datasets/350_images/runs/detect/train4/weights/best.pt')

classNames = ["license-plate", "vehicle"]

#carpeta_fotos = "C:\\Users\\miica\\Desktop\\Uni\\VC\\VC_Practica4\\otsedom.github.io\\VC\\P5\\Coches con matriculas\\Coches con matriculas"
carpeta_fotos = "C:\\Users\\miica\\.conda\\envs\\VC_P1\\P5\\datasets\\350_images\\train\\images"

# Lista de fotos de la carpeta
imagenes = os.listdir(carpeta_fotos)

# Itera sobre cada archivo en la carpeta
for img in imagenes:

    # Verifica si el archivo es una imagen 
    if img.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        # Construye la ruta completa de la imagen
        ruta_imagen = os.path.join(carpeta_fotos, img)

        # Lee la imagen
        imagen = cv2.imread(ruta_imagen)
        imagen_gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)

        # Perform inference on an image
        results = model(imagen, stream=True)   
        
        # Para cada detección
        for r in results:
            boxes = r.boxes
            
            for box in boxes:
                # Contenedor
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values
                
                # Confianza
                confidence = math.ceil((box.conf[0]*100))/100
                print("Confidence --->",confidence)

                # Clase
                cls = int(box.cls[0])
                print("Class name -->", classNames[cls])

                # Traducimos las etiquetas
                if classNames[cls] == "license-plate":
                    classNameTranslate = "matricula"
                else:
                    classNameTranslate = "coche"

                if classNameTranslate == "matricula":
                    # Cambiamos matricula a escala de grises
                    mat = imagen_gris[y1:y2,x1:x2]
                    
                    #
                    _, threshold_image = cv2.threshold(mat, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                    
                    # OCR
                    text = pytesseract.image_to_string(threshold_image ,lang='eng', config='--psm 6 --oem 1')
                    
                    # Filtramos el texto devuelto al formato que debiera tener una matricula
                    text_filtrado = re.search(r'[A-Z]{0,4}\s*\d{0,6}\s*[A-Z]{0,4}\s*', text)
                    if text_filtrado:
                        text_filtrado = text_filtrado.group()
                        if text_filtrado == '':
                            text_filtrado = text
                    # Texto OCR
                    cv2.putText(imagen, text_filtrado, (x1, y2+15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    # Dibuja el contenedor, la clase y el confidence
                    cv2.rectangle(imagen, (x1, y1), (x2, y2), (255, 255, 0), 2)
                    cv2.putText(imagen, str(classNameTranslate) + " --> " + str(confidence), [x1, y1 - 1], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150, 255, 0), 2)
                
                else:
                    cv2.rectangle(imagen, (x1, y1), (x2, y2), (193, 182, 255), 2)
                    cv2.putText(imagen, str(classNameTranslate) + " --> " + str(confidence), [x1, y1 - 1], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 255, 0), 2)

        # Muestra la imagen
        cv2.imshow('Imagen', imagen)

        # Pasa de foto pulsando la tecla una tecla
        if cv2.waitKey(0) == 255 or cv2.waitKey(0) == 83:                
            continue
            
        # Detenemos pulsado ESC
        if cv2.waitKey(50) == 27:
            break
# Destruye ventanas
cv2.destroyAllWindows()

# Detección de Matrículas de Coches utilizando YOLOv8

En esta práctica, nuestro objetivo es implementar un sistema de detección de matrículas de coches utilizando el modelo YOLOv8 para la detección de vehículos.

### Primera parte ###
#### Detección de matrículas usando bordes ####
El proceso de detección consta de varios pasos para lograr obtener una matrícula:

Primero, utilizamos el modelo YOLOv8 para detectar vehículos en la imagen. Si se identifica un coche o un camión (el modelo tiende a confundir coches grandes con camiones), dividimos la celda aproximadamente a la mitad para así acotar el lugar de búsqueda. Seguidamente, detectamos todos los bordes en la región de interés y filtramos por aquellos que tengan forma rectangular y superen cierto área. Este filtrado consta a su vez de varios pasos:

1. **Filtrado de Rectángulos**: Para asegurarnos de que los bordes formen un rectángulo, calculamos el número de bordes obtenidos, buscando que sean cuatro. Así conseguimos quitar los bordes que tengan formas irregulares con menor o mayor número de bordes.

2. **Filtrado por Área**: A su vez, usamos un pequeño umbral para el área y asegurarnos de quitar bordes que sean demasiado pequeños.

5. **Filtrado por Diferencias de X e Y**: Calculamos las diferencias de X e Y de los lados de un rectángulo. Filtramos las matrículas donde estas diferencias son menores a cierto umbral, permitiendo que las fotos tomadas de lado también sean incluidas en las matrículas filtradas. Este es el paso que mayor filtrado realiza y el que más difícil y problemas ha dado para conseguir que el filtrado se ajuste a lo que buscamos. Hemos probado diversos métodos pero finalmente hemos optado por este ya que daba mejores resultados.

### Segunda parte ###
#### Detección de matrículas entrenando el modelo YOLOv8 y reconocimiento de texto ####

En la segunda parte de esta práctica, entrenamos YOLOv8 con 50 épocas y un conjunto de datos de 350 imágenes. El conjunto de datos incluye imágenes de coches con matrículas y las etiquetas "vehicle" y "license-plate". Este conjunto de datos fue obtenido de [Roboflow](https://public.roboflow.com/object-detection/license-plates-us-eu/3/download/yolov8).

Para la lectura de las matrículas, utilizamos la biblioteca pytesseract, que convierte una imagen en una cadena de texto. Cada vez que se detecta una matrícula, usamos pytesseract y realizamos un filtrado previo, ya que la cadena devuelta suele contener caracteres sin sentido provenientes de las distintas partes de la matrícula según su origen.

Finalmente, dibujamos la matrícula en la propia ventana de la imagen para una fácil visualización.
